# Load Tensors

gradient clipping, learning rate scheduling, advanced regularization techniques,
 Bidirectional RNNs, attention mechanisms, transformer architectures, oscillators

In [13]:
import torch

# List of tensor paths
tensor_paths = [
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/EEG_tensor.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/arnold_tongues_rotation_numbers_tensor.pt",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/bandpowers_embedding_net.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/band_power_tensor.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/dspm_tensor.pt",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/fast_fourier_transform_psd_tensor.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/higuchi_fractal_dimensions_tensor.pt",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/Hurst_tensor.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/mfdfa_concatd_tensor.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/mfdfa_tensor.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/short_time_fourier_transform_tensor.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/transfer_entropy_granular_tensor.pt",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/transfer_entropy_hemispheric_avg_input_tensor.pt",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/transfer_entropy_regional_tensor.pt",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/spectral_entropy_tensor.pt",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/spectral_centroids_tensor.pt",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/freq_max_power_tensor.pt",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/spectral_edge_freqs_tensor.pt",
]

# Load the tensors into a dictionary
tensors = {path.split('/')[-1].replace('.pt', ''): torch.load(path) for path in tensor_paths}

In [14]:
# Load each tensor and print its shape
tensor_shapes = {}
for path in tensor_paths:
    tensor_name = path.split("/")[-1].replace(".pt", "").replace(".pth", "")
    data = torch.load(path)
    
    # Check if the loaded data is a state dictionary or a tensor
    if isinstance(data, torch.Tensor):
        tensor_shapes[tensor_name] = data.shape
    elif isinstance(data, dict):  # Likely a state_dict
        tensor_shapes[tensor_name] = "state_dict (model parameters)"
    else:
        tensor_shapes[tensor_name] = "unknown type"

# Print the tensor shapes
for name, shape in tensor_shapes.items():
    print(f"{name}: {shape}")

EEG_tensorh: torch.Size([1, 4227788, 1, 32])
arnold_tongues_rotation_numbers_tensor: torch.Size([32, 300, 300])
bandpowers_embedding_neth: state_dict (model parameters)
band_power_tensorh: torch.Size([4227788, 32, 5])
dspm_tensor: torch.Size([19, 18840, 10])
fast_fourier_transform_psd_tensorh: torch.Size([32, 4227788])
higuchi_fractal_dimensions_tensor: torch.Size([1, 1, 4, 8])
Hurst_tensorh: torch.Size([1, 1, 32, 1])
mfdfa_concatd_tensorh: torch.Size([32, 1, 30, 2])
mfdfa_tensorh: torch.Size([9, 32, 10, 1])
short_time_fourier_transform_tensorh: torch.Size([32, 1001, 4229])
transfer_entropy_granular_tensor: torch.Size([4, 4])
transfer_entropy_hemispheric_avg_input_tensor: torch.Size([92, 92])
transfer_entropy_regional_tensor: torch.Size([4, 4])
spectral_entropy_tensor: torch.Size([1, 1, 32, 1])
spectral_centroids_tensor: torch.Size([1, 1, 32, 1])
freq_max_power_tensor: torch.Size([1, 1, 32, 1])
spectral_edge_freqs_tensor: torch.Size([1, 1, 32, 1])


# Train Test Validation

# CNN

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from apex import amp
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types

class SimplePipeline(Pipeline):
    def __init__(self, batch_size, num_threads, device_id):
        super(SimplePipeline, self).__init__(batch_size, num_threads, device_id, seed=12)
        self.input = ops.FileReader(device="cpu", file_root="/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN")
        self.decode = ops.ImageDecoder(device="cpu", output_type=types.RGB)

    def define_graph(self):
        jpegs, labels = self.input()
        images = self.decode(jpegs)
        return (images, labels)

# Replace these with your data specifics
batch_size = 128
pipe = SimplePipeline(batch_size=batch_size, num_threads=2, device_id=0)
pipe.build()

class EEGPredictor(nn.Module):
    def __init__(self):
        super(EEGPredictor, self).__init__()
        
class BaseEmbeddingNet(nn.Module):
    def __init__(self, input_channels, conv_output_channels, height, width):
        super(BaseEmbeddingNet, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, conv_output_channels, 3)
        self.batch_norm = nn.BatchNorm2d(conv_output_channels)
        self.fc1 = nn.Linear(conv_output_channels * (height - 2) * (width - 2), 512)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.batch_norm(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(self.dropout(x)))
        return x

class EEGEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(1, 64, 4227788, 32)

class RotationEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(32, 64, 300, 300)

class BandPowerEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(32, 64, 4227788, 5)

class DSPMEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(19, 64, 18840, 10)

class FastFourierEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(32, 64, 4227788, 32)

class HiguchiFractalEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(1, 64, 4, 8)

class HurstEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(1, 64, 32, 1)

class MFDFAConcatdEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(32, 64, 30, 2)

class MFDFAEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(9, 64, 32, 10)

class ShortTimeFourierEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(32, 64, 1001, 4229)

class SpectralEntropyEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(1, 64, 32, 1)

class SpectralCentroidsEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(1, 64, 32, 1)

class FreqMaxPowerEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(1, 64, 32, 1)

class SpectralEdgeFreqsEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(1, 64, 32, 1)

class PairwiseMeasureNet(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv = nn.Conv2d(input_channels, output_channels, 1)
    
    def forward(self, x):
        return self.conv(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EEGPredictor()
model = nn.DataParallel(model)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model, optimizer = amp.initialize(model, optimizer, opt_level="O1")

accumulation_steps = 4

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        criterion = nn.CrossEntropyLoss()
        num_epochs = 50

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Use AMP for loss scaling
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        
        # Don't step optimizer for every batch. Only step every accumulation_steps
        if (i+1) % accumulation_steps == 0:             
            optimizer.step()
            optimizer.zero_grad()

# Tranformer Kuramoto RNN

In [ ]:
import torch
from torchdiffeq import odeint
import torch.nn as nn

class CompleteEEGPredictor(nn.Module):
    def __init__(self, d_model, nhead, num_encoder_layers, dim_feedforward):
        super(CompleteEEGPredictor, self).__init__()

        # EEG Embedding
        self.eeg_embedding = EEGEmbeddingNet()
        
class KuramotoODEFunc(nn.Module):
    def __init__(self, omega, K):
        super(KuramotoODEFunc, self).__init__()
        self.omega = omega
        self.K = K

    def forward(self, t, theta):
        N = theta.shape[1]
        dtheta = self.omega + (self.K / N) * torch.sum(torch.sin(theta - theta.unsqueeze(1)), dim=1)
        return dtheta
       class KuramotoLayer(nn.Module):
    def __init__(self, oscillator_count, time_steps, dt=0.01):
        super(KuramotoLayer, self).__init__()
        self.oscillator_count = oscillator_count
        self.time_steps = time_steps
        self.dt = dt
        
        self.omega = nn.Parameter(torch.randn(oscillator_count))
        self.K = nn.Parameter(torch.tensor(1.0))  # global coupling strength

    def forward(self, x):
        ode_func = KuramotoODEFunc(self.omega, self.K)
        time_points = torch.arange(0, self.time_steps * self.dt, self.dt)
        theta = odeint(ode_func, x, time_points)
        return theta
        
        # Transformer model
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward),
            num_encoder_layers
        )
        
        # Positional Encoding (assuming a sequence length of 4227788 for simplicity; adjust as needed)
        self.pos_encoder = nn.Embedding(4227788, d_model) 
        self.position = torch.arange(4227788).unsqueeze(1)
        
        # RNN
        self.rnn = nn.LSTM(input_size=d_model, hidden_size=256, num_layers=2, batch_first=True, dropout=0.5)
        
        # Decoder
        self.fc_out = nn.Linear(256, 32) # 32 channels of EEG data

    def forward(self, src, tgt=None):
        # src is the input EEG sequence, tgt is the target sequence (can be None during inference)
        
        # Embedding
        src_emb = self.eeg_embedding(src)
        
        # Kuramoto Layer
        src_kuramoto = self.kuramoto(src_emb)
        
        # Adding positional encoding
        src_kuramoto = src_kuramoto + self.pos_encoder(self.position).permute(1, 0, 2)

        # Transformer
        transformer_out = self.transformer(src_kuramoto)
        
        # RNN
        rnn_out, _ = self.rnn(transformer_out)
        
        # Decoding the output to the desired size
        return self.fc_out(rnn_out)
